In [2]:
import pandas as pd
from pathlib import Path

YEAR = 2023
BASE = Path(f"Hospitales_raw/Hospitales_{YEAR}")


In [3]:
import pandas as pd
from pathlib import Path

BASE_2023 = Path(".")  # carpeta actual (Hospitales_2023)

print("CWD:", Path.cwd())
print("Archivos:", [p.name for p in BASE_2023.iterdir()])

cat_path  = next(BASE_2023.glob("*Catalogo*.csv"))
ccaa_path = next(BASE_2023.glob("*CCAA*.csv"))
df_path  = next(BASE_2023.glob("*DF*.csv"))


print("Usando:", cat_path.name, ccaa_path.name, df_path.name)


cat  = pd.read_csv(cat_path,  sep=";", encoding="latin1", low_memory=False)
ccaa = pd.read_csv(ccaa_path, sep=";", encoding="latin1", low_memory=False)
pat  = pd.read_csv(df_path,  sep=";", encoding="latin1", low_memory=False)








CWD: c:\Users\nuez_\Desktop\Proyecto ENSE\Hospitales_raw\Hospitales_2023
Archivos: ['05_Hospitales2023.ipynb', 'CNH_Catalogo_2023.csv', 'CNH_CCAA_2023.csv', 'CNH_DF_2023.csv']
Usando: CNH_Catalogo_2023.csv CNH_CCAA_2023.csv CNH_DF_2023.csv


In [4]:
cat.columns = cat.columns.str.strip()
ccaa.columns = ccaa.columns.str.strip()
pat.columns = pat.columns.str.strip()


In [5]:
cat.columns.tolist(), ccaa.columns.tolist(), pat.columns.tolist()

(['CCN',
  'CODCNH',
  'Nombre Centro',
  'Dirección',
  'Teléfono',
  'Cód. Municipio',
  'Municipio',
  'Cód. Provincia',
  'Provincia',
  'Cód. CCAA',
  'CCAA',
  'Código Postal',
  'CAMAS',
  'Cód. Clase de Centro',
  'Clase de Centro',
  'Cód. Dep. Funcional',
  'Dependencia Funcional',
  'Forma parte Complejo',
  'CODIDCOM',
  'Nombre del Complejo',
  'ALTA',
  'Email'],
 ['CODAUTO', 'AUTONOMIA', 'TEXTO_AUTONOMIA'],
 ['COD. DEP. FUNCIONAL', 'DEPENDENCIA FUNCIONAL'])

In [6]:
# 1) Renombrar claves para hacer merges sin líos
cat = cat.rename(columns={
    "Cód. CCAA": "CODAUTO",
    "Cód. Dep. Funcional": "COD_DF",
    "Dependencia Funcional": "Dependencia_DF_raw",
    "CCAA": "CCAA_raw"
})

ccaa = ccaa.rename(columns={
    "CODAUTO": "CODAUTO",
    "AUTONOMIA": "CCAA_nombre"
})

df = pat.rename(columns={
    "COD. DEP. FUNCIONAL": "COD_DF",
    "DEPENDENCIA FUNCIONAL": "Dependencia_DF"
})

# 2) Asegurar tipos iguales para merge (evita el ValueError típico)
cat["CODAUTO"] = pd.to_numeric(cat["CODAUTO"], errors="coerce").astype("Int64")
ccaa["CODAUTO"] = pd.to_numeric(ccaa["CODAUTO"], errors="coerce").astype("Int64")

cat["COD_DF"] = pd.to_numeric(cat["COD_DF"], errors="coerce").astype("Int64")
df["COD_DF"]  = pd.to_numeric(df["COD_DF"],  errors="coerce").astype("Int64")

# 3) Merge CCAA (nombre oficial)
cat = cat.merge(ccaa[["CODAUTO", "CCAA_nombre"]], on="CODAUTO", how="left")

# 4) Merge DF (nombre oficial)
cat = cat.merge(df[["COD_DF", "Dependencia_DF"]], on="COD_DF", how="left")

# 5) Columna año
cat["año"] = 2023

# 6) Titularidad (regla práctica: público si DF contiene “PÚBLIC”, si no -> privado/otros)
dep = cat["Dependencia_DF"].fillna(cat["Dependencia_DF_raw"]).astype(str).str.upper()

cat["titularidad"] = "Otros"
cat.loc[dep.str.contains("PRIV", na=False), "titularidad"] = "Privado"
cat.loc[dep.str.contains("PÚB|PUBLIC", na=False), "titularidad"] = "Público"

# Comprobación rápida
cat["CCAA_nombre"].isna().sum(), len(cat), cat["titularidad"].value_counts(dropna=False)


(np.int64(0),
 845,
 titularidad
 Privado    418
 Otros      374
 Público     53
 Name: count, dtype: int64)

In [7]:
cat["Dependencia_DF"].value_counts(dropna=False).head(30)



Dependencia_DF
Privados                                                               418
Servicios E Institutos De Salud De Las Comunidades Autónomas           322
Otras Entidades U Organismos Públicos                                   40
Mútuas Colaboradoras Con La Seguridad Social                            20
Organizaciones No Gubernamentales                                       20
Otros Centros O Establecimientos Públicos De Dependencia Autonómica     11
Municipio                                                                6
Ministerio De Defensa                                                    2
Instituto De Gestión Sanitaria-Ingesa                                    2
Diputación O Cabildo                                                     2
Otros Centros O Establecimientos Públicos De Dependencia Estatal         2
Name: count, dtype: int64

In [8]:
def clasificar_titularidad_2023(x):
    if pd.isna(x):
        return "Otros"
    x = x.upper()
    if "PRIVAD" in x:
        return "Privado"
    if (
        "COMUNIDAD" in x
        or "SERVICIOS E INSTITUTOS DE SALUD" in x
        or "SEGURIDAD SOCIAL" in x
        or "INGESA" in x
        or "MINISTERIO" in x
        or "DIPUTACIÓN" in x
        or "MUNICIPIO" in x
    ):
        return "Público"
    return "Otros"

cat["titularidad"] = cat["Dependencia_DF"].apply(clasificar_titularidad_2023)




In [9]:
cat["titularidad"].value_counts()





titularidad
Privado    418
Público    354
Otros       73
Name: count, dtype: int64

In [10]:
cat["año"] = 2023



In [11]:
# 1) Normaliza columnas
cat.columns = cat.columns.str.strip()

# 2) Crea CCAA final (texto) desde CCAA_raw
cat["CCAA"] = (
    cat["CCAA_raw"]
    .astype(str)
    .str.strip()
    .str.upper()
)

In [12]:
agg_2023 = (
    cat
    .groupby(["CCAA", "año", "titularidad"], as_index=False)
    .size()
    .rename(columns={"size": "n_hospitales"})
)

totales_2023 = (
    agg_2023
    .groupby(["CCAA", "año"], as_index=False)["n_hospitales"]
    .sum()
    .rename(columns={"n_hospitales": "total_hospitales"})
)

agg_2023 = agg_2023.merge(totales_2023, on=["CCAA", "año"], how="left")
agg_2023["pct"] = agg_2023["n_hospitales"] / agg_2023["total_hospitales"] * 100


In [13]:
agg_2023.head(10)

,CCAA,año,titularidad,n_hospitales,total_hospitales,pct
0,ANDALUCÍA,2023,Otros,1,135,0.740741
1,ANDALUCÍA,2023,Privado,59,135,43.703704
2,ANDALUCÍA,2023,Público,75,135,55.555556
3,ARAGÓN,2023,Otros,4,30,13.333333
4,ARAGÓN,2023,Privado,7,30,23.333333
5,ARAGÓN,2023,Público,19,30,63.333333
6,C. FORAL DE NAVARRA,2023,Privado,6,10,60.000000
7,C. FORAL DE NAVARRA,2023,Público,4,10,40.000000
8,CANARIAS,2023,Otros,2,40,5.000000
9,CANARIAS,2023,Privado,22,40,55.000000


In [14]:
from pathlib import Path

out = Path("../..") / "Data_clean" / "Hospitales_agregado_2023.csv"
agg_2023.to_csv(out, index=False, encoding="utf-8-sig")
out


WindowsPath('../../Data_clean/Hospitales_agregado_2023.csv')

In [15]:
import os
print(os.getcwd())


c:\Users\nuez_\Desktop\Proyecto ENSE\Hospitales_raw\Hospitales_2023


In [16]:
from pathlib import Path

Path(".").resolve(), list(Path(".").iterdir())


(WindowsPath('C:/Users/nuez_/Desktop/Proyecto ENSE/Hospitales_raw/Hospitales_2023'),
 [WindowsPath('05_Hospitales2023.ipynb'),
  WindowsPath('CNH_Catalogo_2023.csv'),
  WindowsPath('CNH_CCAA_2023.csv'),
  WindowsPath('CNH_DF_2023.csv')])

In [17]:
BASE = Path("../../Data_clean")
list(BASE.iterdir())


[WindowsPath('../../Data_clean/ENSE_2006_clean.csv'),
 WindowsPath('../../Data_clean/ENSE_2017_clean.csv'),
 WindowsPath('../../Data_clean/ENSE_2020_clean.csv'),
 WindowsPath('../../Data_clean/ENSE_2023_clean.csv'),
 WindowsPath('../../Data_clean/ENSE_All_clean.csv'),
 WindowsPath('../../Data_clean/Hospitales_agregado_2017.csv'),
 WindowsPath('../../Data_clean/Hospitales_agregado_2020.csv'),
 WindowsPath('../../Data_clean/Hospitales_agregado_2023.csv'),
 WindowsPath('../../Data_clean/Hospitales_agregado_ALL.csv'),
 WindowsPath('../../Data_clean/PIB_CCAA_2017_2020_2023_FINAL.csv')]

In [18]:
h17 = pd.read_csv(BASE / "Hospitales_agregado_2017.csv")
h20 = pd.read_csv(BASE / "Hospitales_agregado_2020.csv")
h23 = pd.read_csv(BASE / "Hospitales_agregado_2023.csv")


In [19]:
print("CWD:", Path(".").resolve())
print("Existe Data_clean:", BASE.exists())



CWD: C:\Users\nuez_\Desktop\Proyecto ENSE\Hospitales_raw\Hospitales_2023
Existe Data_clean: True


In [20]:
hospitales_all = pd.concat([h17, h20, h23], ignore_index=True)

hospitales_all.to_csv(
    BASE / "Hospitales_agregado_ALL.csv",
    index=False,
    encoding="utf-8"
)



In [21]:
hospitales_all.groupby(["año", "titularidad"])["n_hospitales"].sum()




año   titularidad
2017  Otros           19
      Privado        435
      Público        434
2020  Otros           34
      Privado        421
      Público        382
2023  Otros           73
      Privado        418
      Público        354
Name: n_hospitales, dtype: int64

In [22]:
hospitales_all

,CCAA,año,titularidad,n_hospitales,total_hospitales,pct
0,ANDALUCÍA,2017,Otros,1,132,0.760000
1,ANDALUCÍA,2017,Privado,61,132,46.210000
2,ANDALUCÍA,2017,Público,70,132,53.030000
3,ARAGÓN,2017,Otros,1,29,3.450000
4,ARAGÓN,2017,Privado,8,29,27.590000
...,...,...,...,...,...,...
136,PPDO. DE ASTURIAS,2023,Otros,1,26,3.846154
137,PPDO. DE ASTURIAS,2023,Privado,14,26,53.846154
138,PPDO. DE ASTURIAS,2023,Público,11,26,42.307692
139,REGIÓN DE MURCIA,2023,Privado,16,28,57.142857
